In [1]:
%load_ext autoreload

In [2]:
import os
import sys
sys.path.append('../src-py/')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [4]:
%autoreload
import prompted_conclusion_utils as conc_utils

#### Mining important concepts from posts (EXAMPLE):

In [7]:
post = ['while i ve been pro choice for as long as i have been aware of the existence of abortion, i can somewhat sympathize with elements of the pro life mindset.',
 'after all, if you truly believed that a fertilized egg is the same thing as a baby, then naturally you d regard abortion as child murder, and being angry hateful towards perceived murderers of children is pretty understandable.',
 'under that mindset, taking political action to ensure as few abortions as possible is sensible and relatable.',
 'where i lose that sympathy is in seeing the sum total of their political goals, related back to the idea that abortion is essentially child murder.',
 'for example, expanded access to contraceptives and comprehensive sex education are both shown to reduce the rates of both unintended pregnancy and abortion.',
 'if i truly believed that every abortion was a murdered baby, one could reasonably expect me to support pretty much any policy that would decrease the number of abortions, including these two.',
 'yet most of the same people who equate abortion with murder also support policies that restrict sex education and access to contraceptives, which go against the fundamental idea of preventing abortions.',
 'my belief is that pro lifers want fewer abortions only in the context of an overall socially conservative atmosphere, of which fewer abortions is just one small part.',
 'i believe that this worldview is more concerned with not being the sort of society where on demand abortion is legal than it is with the actual abortions themselves.']

In [8]:
post_nps = conc_utils.get_wiki_concepts([post])

TermWikifierClient: 100%|██████████| 36/36 [00:00<00:00, 88.40it/s]


In [9]:
post_nps

[['Social conservatism',
  'Anti-abortion movement',
  'Abortion-rights movements']]

#### Generate multiple conclusions (EXAMPLE):

In [40]:
gen_kwargs = {
    "do_sample": False, 
    "max_length":20,
    "top_p":0.95, 
    "num_beams":1
}

In [41]:
[conc_utils.generate_prompted_conclusions([post],[np], gen_kwargs) for np in post_nps[0] + ['Child murder', 'Sex education']]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

[['Social conservatism is more reasonable than being pro-life'],
 ['Anti-abortion movement is just a bunch of semantics'],
 ['Abortion-rights movements are hypocritical and fall prey to the pro-life mindset'],
 ['Child murder is not a crime, but rather a moral issue'],
 ['Sex education should be included in the policy on abortion.']]

In [16]:
post = ['i find most rap so incredibly mindnumbingly boring',
 "it's mostly repetitive beats with some poetry rapped over it and maybe a sung chorus in it for good measure",
 "problem is that for the most part it just lacks any real sense of melody and the rythms are incredibly basic 4 4 beats that never change throughout the song and as such there really isn't anything interesting or earcatching to latch onto musically",
 "amp; x200b; i've heard some people say that rap is supposed to be about the lyrics and not the music, but that just begs the question why it couldn't be published as some poetry record or spoken word album if the music simply is just filler to get some poetry out there",
 'of course there are some exceptions to this, some rap artists have more interesting beats that actually have something going on musically, the flow changes depending on the mood that section of the song is trying to convey and sometimes they just use a real band for the actual music but this feels like a rarity for the genre in general',
 "most of the lyrics also feel really basic and childish, even some of the more 'deeper' and well written lyrics that others recommend me just end up boiling down to the most baseline social critiques and an unhealthy obsession with penises in some tired and childish sexual metaphor",
 "amp; x200b; i've even heard people say that it's not about the music or the lyrics that's the point in some of these songs but that they try to get you pumped up and hype but i don't really find that kinda music very hype inducing, especially when it's just some simple beat that my dj friend could throw together in ten minutes that they just mumble or lazily yell over",
 "even the live shows are lame with the actual album track, vocals and all, being used as the backtrack while the rapper kinda half hardheartedly yells or mumbles and tries to get the crowd to be hyped, that's not hype, screaming into a guitar or bass mic on the fly to get a weird vocal effect is hype, screaming your lungs out to be heard over the band backtrack cause the mic broke is hype, having just the album track playing is not hype, it's lazy and it feels like people are just celebrating mediocrity"]

In [17]:
conc_utils.generate_prompted_conclusions([post],['Hip hop music'], gen_kwargs)

  0%|          | 0/1 [00:00<?, ?ba/s]

['Hip hop music is a really boring, uninteresting music, and should not be crit']

### Generate multiple conclusions for development dataset:

In [13]:
#sample_df = valid_df[valid_df.post.str.len() < 50].sample(10)

In [5]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":20,
    "top_p":0.95, 
    "num_beams":1
}

def generate_multiple_conclusions(df):
    posts_conclusions = []
    for i in range(6):
        control_codes = [x[i] if len(x) > i else '' for x in df['post_concepts'].tolist()]
        conclusions   = conc_utils.generate_prompted_conclusions(df['post'].tolist(), control_codes, gen_kwargs, batch_size=2)
        posts_conclusions.append(zip(control_codes, conclusions))

    df['post_conclusions'] = list(map(list, zip(*posts_conclusions))) # transpose
    
    return df

In [6]:
df['post_concepts'] = conc_utils.get_wiki_concepts(df['post'].tolist(), max_concepts=5)
df = generate_multiple_conclusions(df)

NameError: name 'df' is not defined

In [ ]:
df[['title', 'post', 'post_concepts', 'post_conclusions']].head(n=100)

In [31]:
df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_multi_conclusions.pkl')

1499

### Run on Test:

In [ ]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed.pkl')

In [ ]:
test_df['post_concepts'] = conc_utils.get_wiki_concepts(test_df['post'].tolist(), max_concepts=5)

TermWikifierClient: 100%|██████████| 67/67 [00:00<00:00, 77.86it/s]

In [7]:
test_df = generate_multiple_conclusions(test_df)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
test_df[['title', 'post', 'post_concepts']].head(n=5)

title  \
500025                                                                                                    I Don't Believe in the Big Bang   
534021                                                                                There is no reason for Britain to remain in the EU.   
415645   The Trump Administration Would Be Significantly More Harmful to US Domestic and International Policy W/out Moderates Like Mattis   
507298                             Poisoning the well against fascists is a dishonest and therefore ineffective way of combating fascists   
505763                                                                       Women have an easier time finding romantic partners than men   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [9]:
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl')

In [10]:
print('doneeeee')

doneeeee


### Manually evaluate whether prompt-conclusions are valid conclusions:

In [42]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl')

In [43]:
sample_test = test_df.sample(50)

In [44]:
sample_test['conclusions'] = sample_test.post_conclusions.apply(lambda concs: [x[1] for x in concs if x[0] != ''])

In [53]:
sample_test[['post', 'title', 'conclusions']].head(n=10)

post  \
462494                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [29]:
sample_test[['post', 'title', 'conclusions']].to_csv('../data/prompted-conclusion-sample.csv')